In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
# from shapely.ops import split, snap, nearest_points
import networkx as nx

# import netana as na

In [2]:
network_crs = "3857"
df_streets = gpd.read_file("./notebook/street_resistencia_corrientes_v3.shp")
df_streets = df_streets.dissolve()

# Create network GeoDataFrame
- id, length and weight field
- optionally a label field
- defined crs 

In [3]:
from netana.graph_manager.network import split_intersecting_lines, create_gdf_network

# topology
df_streets = split_intersecting_lines(df_streets)
# create network
gdf_network = create_gdf_network(df_streets, network_crs, v_speed=4.5, v_label='calle')
print(gdf_network.crs)
print(len(gdf_network))
gdf_network.head()

EPSG:3857
47


,id,length,weight,label,geometry
0,e_1,458.76,6.116800,calle,"LINESTRING (-6546884.300 -3184165.694, -654703..."
1,e_2,459.53,6.127067,calle,"LINESTRING (-6546859.721 -3184019.784, -654700..."
2,e_3,460.80,6.144000,calle,"LINESTRING (-6547295.180 -3183823.596, -654714..."
3,e_4,471.56,6.287467,calle,"LINESTRING (-6546820.169 -3183766.855, -654695..."
4,e_5,461.49,6.153200,calle,"LINESTRING (-6547241.580 -3183488.333, -654711..."


# Create LINES and POINTS DATASETS; and GRAPH

In [4]:
from netana.graph_manager.graph_creation import graph_from_gdf

G, gdf_nodes, gdf_edges  = graph_from_gdf(gdf_network)

gdf_edges.to_file("./output/pruebas_2025/edges.shp") # TO DO - save and load to db
gdf_nodes.to_file("./output/pruebas_2025/nodes.shp")
G

# SOURCES

In [5]:
from netana.graph_manager.sources import create_sources, sources_to_graph, create_sources_edges_and_nodes

sources_file_path = "./notebook/source_corrientes.shp"
gdf_locations = gpd.read_file(sources_file_path)
gdf_locations = gdf_locations.explode(index_parts=True).reset_index(drop=True)

gdf_locations = create_sources(gdf_locations, network_crs, v_label='plaza')
gdf_locations

,id,label,geometry
0,1,plaza,"POLYGON ((-6546874.064 -3184160.022, -6546867...."
1,2,plaza,"POLYGON ((-6546624.137 -3183661.595, -6546603...."


In [6]:
G_sources, gdf_edges_sources, gdf_nodes_sources, gdf_locations = sources_to_graph(G, gdf_edges, gdf_nodes, gdf_locations, search_radio=10)
gdf_edges_sources.to_file("./output/pruebas_2025/edges.shp")
gdf_nodes_sources.to_file("./output/pruebas_2025/nodes.shp")
print(gdf_locations.crs)
gdf_locations.head()

EPSG:3857


c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\geopandas\geodataframe.py:1538: SettingW

,id,label,geometry,node_id
0,1,plaza,"POLYGON ((-6546874.064 -3184160.022, -6546867....",1_cen
1,2,plaza,"POLYGON ((-6546624.137 -3183661.595, -6546603....",2_cen


# ISOCHRONES

In [18]:
from netana.isochrones.isochrones import create_isochrones_multiple_weights

gdf_iso = create_isochrones_multiple_weights(
    G_sources, gdf_nodes_sources, gdf_edges_sources, gdf_locations, "weight" , [1,2,3],  50, 50, 
    f_dissolve='weight_limit', area_type='rings', v_crs_proj="3857")
gdf_iso.to_file("./output/pruebas_2025/edges_iso.shp")
gdf_iso

C:\Users\Compumar\AppData\Local\Temp\ipykernel_1916\1027097683.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_iso.to_file("./output/pruebas_2025/edges_iso.shp")


,weight_limit,geometry
0,2,"MULTIPOLYGON (((-6546517.353 -3183635.602, -65..."
1,3,"MULTIPOLYGON (((-6546855.304 -3183645.587, -65..."
2,2,"MULTIPOLYGON (((-6546474.721 -3183647.875, -65..."
3,1,"MULTIPOLYGON (((-6546667.844 -3183696.786, -65..."


In [19]:
from netana.isochrones.isochrones import create_individual_isochrones_multiple_weights

gdf_iso_ind = create_individual_isochrones_multiple_weights(
    G_sources, gdf_edges_sources, gdf_locations, "weight", [1,2,3], 50, 50, 
    f_dissolve='weight_limit', area_type='rings', v_frac=0, v_time_factor= 20)
gdf_iso_ind.to_file("./output/pruebas_2025/edges_iso_inv.shp")
gdf_iso_ind

C:\Users\Compumar\AppData\Local\Temp\ipykernel_1916\3702680235.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_iso_ind.to_file("./output/pruebas_2025/edges_iso_inv.shp")


,weight_limit,geometry,node_source_st
0,2,"POLYGON ((-6546843.191 -3184232.487, -6546842....",NaN
1,3,"MULTIPOLYGON (((-6546717.154 -3184097.713, -65...",NaN
2,3,"POLYGON ((-6546753.779 -3183569.723, -6546753....",NaN
3,2,"POLYGON ((-6546843.199 -3184232.545, -6546842....",NaN
4,2,"MULTIPOLYGON (((-6546517.353 -3183635.602, -65...",NaN
5,3,"MULTIPOLYGON (((-6546688.781 -3184165.273, -65...",NaN
6,3,"MULTIPOLYGON (((-6546855.304 -3183645.587, -65...",NaN
7,2,"POLYGON ((-6546843.199 -3184232.545, -6546842....",NaN
8,2,"MULTIPOLYGON (((-6546517.353 -3183635.602, -65...",NaN
9,2,"MULTIPOLYGON (((-6546925.230 -3184107.670, -65...",NaN
